<a href="https://colab.research.google.com/github/hniu-tamu/keras/blob/main/keras_tuner/keras_tuner_a_quick_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.7 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [3]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0
print("x_train.shape:", x_train.shape)

11490434/11490434 [==============================] - 1s 0us/step
x_train.shape: (60000, 28, 28)


In [4]:
x_test.shape

(10000, 28, 28)

In [5]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

<ipython-input-5-7bb571729a7c>:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [6]:
def build_model(hp):
  model = keras.Sequential()
  model.add(layers.Flatten(input_shape = (28, 28)))

  model.add(layers.Dense(units = hp.Int('units',
                                        min_value = 32,
                                        max_value = 512,
                                        step = 128),
                         activation = 'relu'))
  model.add(layers.Dense(10, activation = 'softmax'))
  model.compile(
      optimizer = keras.optimizers.Adam(
          hp.Choice('learning_rate',
                    values = [1e-2, 1e-4])),
      loss = 'sparse_categorical_crossentropy',
      metrics = ['accuracy']
  )
  return model

In [7]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials = 5,
    executions_per_trial = 3,
    directory = 'my_dir',
    project_name = 'helloworld'
)

In [8]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 128, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.0001], 'ordered': True}


In [9]:
tuner.search(x_train, y_train,
             epochs = 5,
             validation_data=(x_test, y_test))

Trial 5 Complete [00h 01m 56s]
val_accuracy: 0.9562666614850363

Best val_accuracy So Far: 0.9620999892552694
Total elapsed time: 00h 09m 21s


In [10]:
tuner.results_summary()

Results summary
Results in my_dir/helloworld
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
units: 416
learning_rate: 0.01
Score: 0.9620999892552694

Trial 0 summary
Hyperparameters:
units: 288
learning_rate: 0.0001
Score: 0.9616333246231079

Trial 1 summary
Hyperparameters:
units: 160
learning_rate: 0.01
Score: 0.9607999920845032

Trial 4 summary
Hyperparameters:
units: 32
learning_rate: 0.01
Score: 0.9562666614850363

Trial 3 summary
Hyperparameters:
units: 160
learning_rate: 0.0001
Score: 0.9534333348274231


In [17]:
def build_model2(hp):
  model = keras.Sequential()
  model.add(layers.Flatten(input_shape = (28, 28)))

  for i in range(hp.Int('layers', 2, 6)):
    model.add(layers.Dense(
        units = hp.Int('units_' + str(i), 50, 100, step = 10),
        activation = hp.Choice('act_' + str(i), ['relu', 'sigmoid'])))

  model.add(layers.Dense(10, activation = 'softmax'))
  model.compile('adam', 'sparse_categorical_crossentropy', metrics = ['accuracy'])
  return model

In [18]:
tuner2 = RandomSearch(
    build_model2,
    objective='val_accuracy',
    max_trials = 4,
    executions_per_trial = 1,
    directory = 'my_dir2',
    project_name = 'helloworld'
)

In [19]:
tuner2.search_space_summary()

Search space summary
Default search space size: 5
layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 6, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 50, 'max_value': 100, 'step': 10, 'sampling': 'linear'}
act_0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid'], 'ordered': False}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 50, 'max_value': 100, 'step': 10, 'sampling': 'linear'}
act_1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid'], 'ordered': False}


In [20]:
tuner2.search(x_train, y_train,
              epochs = 5,
              validation_data= (x_test, y_test))

Trial 4 Complete [00h 00m 44s]
val_accuracy: 0.9552000164985657

Best val_accuracy So Far: 0.9760000109672546
Total elapsed time: 00h 02m 47s


In [21]:
tuner2.results_summary()

Results summary
Results in my_dir2/helloworld
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
layers: 2
units_0: 80
act_0: relu
units_1: 100
act_1: relu
units_2: 80
act_2: sigmoid
units_3: 60
act_3: sigmoid
units_4: 70
act_4: relu
Score: 0.9760000109672546

Trial 0 summary
Hyperparameters:
layers: 5
units_0: 70
act_0: sigmoid
units_1: 60
act_1: relu
units_2: 50
act_2: relu
units_3: 50
act_3: relu
units_4: 50
act_4: relu
Score: 0.9717000126838684

Trial 2 summary
Hyperparameters:
layers: 6
units_0: 90
act_0: relu
units_1: 70
act_1: sigmoid
units_2: 100
act_2: relu
units_3: 60
act_3: sigmoid
units_4: 80
act_4: sigmoid
units_5: 50
act_5: relu
Score: 0.9710999727249146

Trial 3 summary
Hyperparameters:
layers: 6
units_0: 50
act_0: sigmoid
units_1: 90
act_1: sigmoid
units_2: 60
act_2: relu
units_3: 80
act_3: relu
units_4: 100
act_4: relu
units_5: 60
act_5: sigmoid
Score: 0.9552000164985657


In [22]:
# Delete the non-empty folder with shutil

# import shutil
# shutil.rmtree('my_dir2')